https://github.com/jupyter/notebook/issues/2287

incase you run into an issue with running the code due to exceeding run rate (?)

In [1]:
# imports for database connection
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine, func

# import these to view table column headers & rows
from sqlalchemy.orm import sessionmaker
from sqlalchemy import select

# Import the dependencies for analysis
%matplotlib inline

import numpy as np
import pandas as pd
import datetime as dt
from datetime import date
from datetime import timedelta
import matplotlib.pyplot as plt

In [2]:
# UPDATE PATH ONCE FINAL ANALYSIS FILE IS COMPILED
engine = create_engine("sqlite:///../../Resources/covid_db.db")

In [3]:
# reflect an existing database into a new model (creates base class for automap schema)
Base = automap_base()

In [4]:
# reflect the tables
Base.prepare(engine, reflect=True)

In [5]:
Base.classes.keys()

['covid_stats', 'government_regulation']

In [6]:
# save references to each table
covid_stats = Base.classes.covid_stats
government_regulation = Base.classes.government_regulation

In [7]:
# Create session (link) from Python to the database
session = sessionmaker(bind=engine)()

In [8]:
# perform a query to retrieve the data
results = session.query(government_regulation.primary_key_response,
                       government_regulation.Date_updated,
                       government_regulation.CountryName,
                       government_regulation.CountryCode,
                       government_regulation.StringencyIndex_updated,
                       government_regulation.percentage_change_stringency,
                       government_regulation.StringencyLegacyIndex_updated,
                       government_regulation.GovernmentResponseIndex_updated,
                       government_regulation.percentage_change_GovernmentResponse,
                       government_regulation.ContainmentHealthIndex_updated,
                       government_regulation.percentage_change_ContainmentHealth ,
                       government_regulation.EconomicSupportIndex_updated,
                       government_regulation.percentage_change_EconomicSupport
                       ).all()
    
# save the query results as a Pandas DataFrame and set the index
government_regulation = pd.DataFrame(results, columns=['primary_key_response',
                                                       'Date_updated',
                                                       'CountryName',
                                                       'CountryCode',
                                                       'StringencyIndex_updated',
                                                       'percentage_change_stringency',
                                                       'StringencyLegacyIndex_updated',
                                                       'GovernmentResponseIndex_updated',
                                                       'percentage_change_GovernmentResponse',
                                                       'ContainmentHealthIndex_updated',
                                                       'percentage_change_ContainmentHealth',
                                                       'EconomicSupportIndex_updated',
                                                       'percentage_change_EconomicSupport'
                                                      ])

In [9]:
# preview 
government_regulation

,primary_key_response,Date_updated,CountryName,CountryCode,StringencyIndex_updated,percentage_change_stringency,StringencyLegacyIndex_updated,GovernmentResponseIndex_updated,percentage_change_GovernmentResponse,ContainmentHealthIndex_updated,percentage_change_ContainmentHealth,EconomicSupportIndex_updated,percentage_change_EconomicSupport
0,ABW2020-01-01,2020-01-01,Aruba,ABW,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0
1,ABW2020-01-02,2020-01-02,Aruba,ABW,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0
2,ABW2020-01-03,2020-01-03,Aruba,ABW,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0
3,ABW2020-01-04,2020-01-04,Aruba,ABW,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0
4,ABW2020-01-05,2020-01-05,Aruba,ABW,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43671,ZWE2020-08-27,2020-08-27,Zimbabwe,ZWE,80.56,0.0,88.1,66.03,0.0,73.48,0.0,25.0,0.0
43672,ZWE2020-08-28,2020-08-28,Zimbabwe,ZWE,80.56,0.0,88.1,66.03,0.0,73.48,0.0,25.0,0.0
43673,ZWE2020-08-29,2020-08-29,Zimbabwe,ZWE,80.56,0.0,88.1,66.03,0.0,73.48,0.0,25.0,0.0
43674,ZWE2020-08-30,2020-08-30,Zimbabwe,ZWE,80.56,0.0,88.1,66.03,0.0,73.48,0.0,25.0,0.0


# Building blocks for for loop

In [ ]:
China = pd.DataFrame(government_regulation.loc[government_regulation["CountryName"]=="China"].reset_index()).drop(["index"],axis=1)

In [ ]:
China["StringencyIndex_updated"].max()

In [ ]:
# Filter based on percentage changes greater than 10% in policy - CANADA
China = China.loc[(China.StringencyIndex_updated >= 10) &
                   (China.percentage_change_stringency > 0) &
                   (China.StringencyIndex_updated <= China["StringencyIndex_updated"].max())
                  ]
China['Date_updated'] = pd.to_datetime(China['Date_updated'])
China.reset_index(inplace=True)

In [ ]:
China

In [ ]:
ramp_up_max = China.StringencyIndex_updated.max()
ramp_up_end_date = China.loc[China.StringencyIndex_updated == ramp_up_max]
ramp_up_end_date = ramp_up_end_date.drop_duplicates(subset=['StringencyIndex_updated'], keep='first')
ramp_up_date = ramp_up_end_date['Date_updated']
index = ramp_up_date.index.values
index = index.tolist()

In [ ]:
index

In [ ]:
China.drop(China.index[np.where(China.index > index)[0]])

In [ ]:
number_of_days = Zimbabwe['Date_updated'].max() - Zimbabwe['Date_updated'].min()
number_of_days.days

# for loop trial 1

In [10]:
countries = government_regulation["CountryName"].unique()

In [11]:
countries

array(['Aruba', 'Afghanistan', 'Angola', 'Anguilla', 'Albania', 'Andorra',
       'United Arab Emirates', 'Argentina', 'Australia', 'Austria',
       'Azerbaijan', 'Burundi', 'Belgium', 'Benin', 'Burkina Faso',
       'Bangladesh', 'Bulgaria', 'Bahrain', 'Bosnia and Herzegovina',
       'Belarus', 'Belize', 'Bermuda', 'Bolivia', 'Brazil', 'Barbados',
       'Brunei', 'Bhutan', 'Botswana', 'Central African Republic',
       'Canada', 'Switzerland', 'Chile', 'China', "Cote d'Ivoire",
       'Cameroon', 'Democratic Republic of Congo', 'Congo', 'Colombia',
       'Cape Verde', 'Costa Rica', 'Cuba', 'Cayman Islands', 'Cyprus',
       'Czech Republic', 'Germany', 'Djibouti', 'Dominica', 'Denmark',
       'Dominican Republic', 'Algeria', 'Ecuador', 'Egypt', 'Eritrea',
       'Spain', 'Estonia', 'Ethiopia', 'Finland', 'Fiji',
       'Falkland Islands', 'France', 'Gabon', 'United Kingdom', 'Georgia',
       'Ghana', 'Gibraltar', 'Guinea', 'Gambia', 'Greece', 'Greenland',
       'Guatemala', 'Gu

In [12]:
len(countries)

179

In [13]:
index_1 = round(len(countries)/4)
index_2 = index_1 + index_1
index_3 = index_2 + index_1

In [14]:
index_1

45

In [15]:
countries11 = countries[:2]
countries1 = countries[:index_1]
countries2 = countries[index_1:index_2]
countries3 = countries[index_2:index_3]
countries4 = countries[index_3:]

In [16]:
countries11

array(['Aruba', 'Afghanistan'], dtype=object)

In [ ]:
countries1

In [ ]:
countries2

In [ ]:
countries3

In [ ]:
countries4

In [17]:
# List of filtered countries for "ramp up" period - stringency index is at minimum 10 to first time index hits its maximum
ramp_up = []

In [18]:
# Loop through all countries and capture ramp up period
for location in countries11:
    country = government_regulation.loc[government_regulation["CountryName"]==location].copy()
   
    country = country.loc[(country.StringencyIndex_updated >= 10) &
                   (country.percentage_change_stringency > 0) &
                   (country.StringencyIndex_updated <= country["StringencyIndex_updated"].max())
                  ]
    
    country['Date_updated'] = pd.to_datetime(country['Date_updated'])
    country.reset_index(inplace=True)
    
    ramp_up_max = country.StringencyIndex_updated.max()
    ramp_up_end_date = country.loc[country.StringencyIndex_updated == ramp_up_max]
    ramp_up_end_date = ramp_up_end_date.drop_duplicates(subset=['StringencyIndex_updated'], keep='first')
    ramp_up_date = ramp_up_end_date['Date_updated']
    index = ramp_up_date.index.values
    index = index.tolist()
    
    country.drop(country.index[np.where(country.index > index)[0]])
    
    ramp_up.append(country)

ramp_up_df = pd.DataFrame()
# del government_regulation
for df in ramp_up:
    ramp_up_df = ramp_up.append(df)

MemoryError: 

In [ ]:
ramp_up_df

# without for loop

In [ ]:
China = government_regulation.loc[government_regulation["CountryName"]=="China"].copy()
   
China = China.loc[(China.StringencyIndex_updated >= 10) &
               (China.percentage_change_stringency > 0) &
               (China.StringencyIndex_updated <= China["StringencyIndex_updated"].max())
              ]
    
China['Date_updated'] = pd.to_datetime(China['Date_updated'])
China.reset_index(inplace=True)
    
ramp_up_max = China.StringencyIndex_updated.max()
ramp_up_end_date = China.loc[China.StringencyIndex_updated == ramp_up_max]
ramp_up_end_date = ramp_up_end_date.drop_duplicates(subset=['StringencyIndex_updated'], keep='first')
ramp_up_date = ramp_up_end_date['Date_updated']
index = ramp_up_date.index.values
index = index.tolist()
    
China.drop(China.index[np.where(China.index > index)[0]])